# Install/import

# What is about ? 

**Briefly:** Example how to work with huge .h5 files without loading them in RAM. 
See also example in https://www.kaggle.com/code/alexandervc/use-h5py-for-huge-h5-file-backing-it-on-disk 

**Data** from the competition , but denoising preprocessing by MAGIC package (https://github.com/KrishnaswamyLab/MAGIC) has been done by Liza Geraseva. See discussion: https://www.kaggle.com/competitions/open-problems-multimodal/discussion/350856

**Work principle:** The main data can be accessed as f['0']['block0_values'] - without loading to RAM, but  when one takes a slice: f['0']['block0_values'][:,IX] - it is converted to numpy array (and loaded into memory?). 
There are some details: one can do constant slices like  f['0']['block0_values'][:10,:10], but to make a  variable slice one need to process in two steps: f['0']['block0_values'][:,IX1][IX0]. 

**Details:** We load the data. And as an exercise and sanity check we produce plots for cell proliferation cycle visualization - so-called G1/S-G2/M plots. The results are as expected - the "cyclic" structes is better seen comparing as to original files (see version 3 of the present notebook). Remark: one can compare with another (more simple denoising ("pooling")) here: https://www.kaggle.com/code/alexandervc/mmscel-cell-cycle-03-daybydaychange-megakaryocyte#G1/S-G2/M-plots-to-analyse-the-cell-cycle

**Context:** See paper: https://arxiv.org/abs/2208.05229 for cell cycle analysis. "Computational challenges of cell cycle analysis using single cell transcriptomics" Alexander Chervov, Andrei Zinovyev. All the work for that paper has been done on Kaggle - see datasets and code therein: https://www.kaggle.com/alexandervc/datasets?scroll=true https://www.kaggle.com/andreizinovyev Hundreds of single cell RNA seq datasets were analyzed.



### Remarks: 
    
    ( in Vesrstion 1- 6 we look mainly on Megakaryocyte Progenitors - just almost smallest and representative (from previous analysis ) 

### Versions:

#### 11 with MAGIC again 

#### 10 - for comparaison - same data as 9, but NO MAGIC 


#### 9 - added plots all cell types
    
    Same MAGIC CITEseq TEST

#### 7,8 cosmetic changes


#### 6 - return to MAGIC - same as version 4 - "test" part of CITE-seq (features)
        

#### 5 - for comparaison  NO(!) MAGIC - difference is STRIKING

    For days 2,4 - we cannot see clear "cyclic" structures without denoising.
    Compare with denoised versions - where "cyclic" structure can be clearly seen ! (E.g. version 4) 
    "test" part of CITE-seq (features)


#### 4 - MAGIC on "test" part of CITE-seq (features)

    "Cyclic" structure is seen VERY well for many days - may be because only one donor, but for "train" part we have 3 donors and they mix and create less clear picture

#### 3 - for comparaison look on original input file

    The "cyclic" structure is less clearly seen for original files, comparing to denoised - as expected. 

#### 1,2 - work with MAGIC denoised data. 



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import time
t0start = time.time()

import pandas as pd
import numpy as np
import os
import sys

import matplotlib.pyplot as plt
#plt.style.use('dark_background')
import seaborn as sns

#If you see a urllib warning running this cell, go to "Settings" on the right hand side, 
#and turn on internet. Note, you need to be phone verified.
!pip install --quiet tables


import h5py
!pip install hdf5plugin~=2.0 # https://forum.hdfgroup.org/t/cant-open-directory-usr-local-hdf5-lib-plugin/9738/4
import hdf5plugin

# !pip install scanpy
# import scanpy as sc
# import anndata

DATA_DIR = "/kaggle/input/open-problems-multimodal/"
FP_CELL_METADATA = os.path.join(DATA_DIR,"metadata.csv")

FP_CITE_TRAIN_INPUTS = os.path.join(DATA_DIR,"train_cite_inputs.h5")
FP_CITE_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_cite_targets.h5")
FP_CITE_TEST_INPUTS = os.path.join(DATA_DIR,"test_cite_inputs.h5")

FP_MULTIOME_TRAIN_INPUTS = os.path.join(DATA_DIR,"train_multi_inputs.h5")
FP_MULTIOME_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_multi_targets.h5")
FP_MULTIOME_TEST_INPUTS = os.path.join(DATA_DIR,"test_multi_inputs.h5")

FP_SUBMISSION = os.path.join(DATA_DIR,"sample_submission.csv")
FP_EVALUATION_IDS = os.path.join(DATA_DIR,"evaluation_ids.csv")

df_cell = pd.read_csv(FP_CELL_METADATA)
#df_cell

# "Load" data and work with it 

In [ ]:
print('Look on Features: ')
filename = '/kaggle/input/citeseq-denoised/test_cite_inputs_denoised.h5'
filename = '/kaggle/input/citeseq-denoised/train_cite_inputs_denoised.h5'
#filename = '/kaggle/input/open-problems-multimodal/train_multi_inputs.h5'
#filename = '/kaggle/input/open-problems-multimodal/train_multi_inputs.h5'
filename = '/kaggle/input/open-problems-multimodal/train_cite_inputs.h5'

#filename = '/kaggle/input/citeseq-denoised/test_cite_inputs_denoised.h5'

#filename = '/kaggle/input/open-problems-multimodal/test_cite_inputs.h5'

f2 = h5py.File(filename,'r')#, mode)

In [ ]:
print(f2.keys() )

In [ ]:
str_data_inf = ''
if filename == '/kaggle/input/open-problems-multimodal/train_cite_inputs.h5':
    first_key = 'train_cite_inputs'
    str_data_inf = ' CITEseq Train '
# elif filename == '/kaggle/input/open-problems-multimodal/train_multi_inputs.h5':
#     first_key = 'train_multi_inputs'
#     str_data_inf = ' Multi Train Features '
elif filename == '/kaggle/input/open-problems-multimodal/test_cite_inputs.h5':
    first_key = 'test_cite_inputs'#'train_multi_inputs' 
    str_data_inf = ' CITEseq Test '
elif filename == '/kaggle/input/citeseq-denoised/test_cite_inputs_denoised.h5':
    first_key = '0'
    str_data_inf = ' MAGIC CITEseq Test '
elif filename == '/kaggle/input/citeseq-denoised/train_cite_inputs_denoised.h5':
    first_key = '0'
    str_data_inf = ' MAGIC CITEseq Train '
# else:
#     first_key = '0'#'train_multi_inputs' 
    
print( f2[first_key].keys() )

In [ ]:
for k in f2[first_key].keys():
    print(type(f2[first_key][k]), f2[first_key][k].shape)

In [ ]:
f2[first_key]['axis0'][0], f2[first_key]['axis1'][0], f2[first_key]['block0_items'][0],  f2[first_key]['block0_values'][0,0], 

In [ ]:
type(f2[first_key]['block0_values'][:10,:10])

In [ ]:
print( dir(f2[first_key]['block0_values'][:10,:10]) )

In [ ]:
list_genes_ensembl_ids = [t.decode().split('_')[0] for t in f2[first_key]['axis0'] ]
print( len(list_genes_ensembl_ids), list_genes_ensembl_ids[:10] )
list_genes_symbols = [t.decode().split('_')[1] for t in f2[first_key]['axis0'] ]
print( len(list_genes_symbols), list_genes_symbols[:10] )

In [ ]:
list_cells_barcodes = [t.decode() for t in f2[first_key]['axis1'] ]
print(len(list_cells_barcodes) , list_cells_barcodes[:10] )

In [ ]:
print( type(f2[first_key]['block0_values']), dir(f2[first_key]['block0_values'] ) )

# G1/S-G2/M cell proliferation cycle plot for all cells 

It will not be very clear - we need to separate days and cell types - see next subsections.

In [ ]:

G1S_genes_Tirosh = ['ENSMUSG00000005410', 'ENSMUSG00000027342', 'ENSMUSG00000025747', 'ENSMUSG00000024742', 'ENSMUSG00000002870', 'ENSMUSG00000022673', 'ENSMUSG00000030978', 'ENSMUSG00000029591', 'ENSMUSG00000031821', 'ENSMUSG00000026355', 'ENSMUSG00000055612', 'ENSMUSG00000037474', 'ENSMUSG00000025395', 'ENSMUSG00000001228', 'ENSMUSG00000031629', 'ENSMUSG00000025001', 'ENSMUSG00000023104', 'ENSMUSG00000028884', 'ENSMUSG00000028693', 'ENSMUSG00000030346', 'ENSMUSG00000006715', 'ENSMUSG00000027242', 'ENSMUSG00000004642', 'ENSMUSG00000028212', 'ENSMUSG00000041712', 'ENSMUSG00000030726', 'ENSMUSG00000024151', 'ENSMUSG00000022360', 'ENSMUSG00000027323', 'ENSMUSG00000020649', 'ENSMUSG00000000028', 'ENSMUSG00000017499', 'ENSMUSG00000039748', 'ENSMUSG00000032397', 'ENSMUSG00000022422', 'ENSMUSG00000030528', 'ENSMUSG00000028282', 'ENSMUSG00000028560', 'ENSMUSG00000042489', 'ENSMUSG00000006678', 'ENSMUSG00000022945', 'ENSMUSG00000034329', 'ENSMUSG00000046179']
G2M_genes_Tirosh = ['ENSMUSG00000054717', 'ENSMUSG00000019942', 'ENSMUSG00000027306', 'ENSMUSG00000001403', 'ENSMUSG00000017716', 'ENSMUSG00000027469', 'ENSMUSG00000020914', 'ENSMUSG00000024056', 'ENSMUSG00000062248', 'ENSMUSG00000026683', 'ENSMUSG00000028044', 'ENSMUSG00000031004', 'ENSMUSG00000019961', 'ENSMUSG00000026605', 'ENSMUSG00000037313', 'ENSMUSG00000020808', 'ENSMUSG00000034349', 'ENSMUSG00000032218', 'ENSMUSG00000048327', 'ENSMUSG00000037725', 'ENSMUSG00000020897', 'ENSMUSG00000027379', 'ENSMUSG00000012443', 'ENSMUSG00000015749', 'ENSMUSG00000036752', 'ENSMUSG00000022385', 'ENSMUSG00000024795', 'ENSMUSG00000044783', 'ENSMUSG00000023505', 'ENSMUSG00000020737', 'ENSMUSG00000006398', 'ENSMUSG00000038379', 'ENSMUSG00000044201', 'ENSMUSG00000028678', 'ENSMUSG00000022391', 'ENSMUSG00000038252', 'ENSMUSG00000037544', 'ENSMUSG00000048922', 'ENSMUSG00000028873', 'ENSMUSG00000027699', 'ENSMUSG00000032254', 'ENSMUSG00000020330', 'ENSMUSG00000027496', 'ENSMUSG00000068744', 'ENSMUSG00000036777', 'ENSMUSG00000004880', 'ENSMUSG00000040549', 'ENSMUSG00000045328', 'ENSMUSG00000005698', 'ENSMUSG00000026622', 'ENSMUSG00000035293', 'ENSMUSG00000074802', 'ENSMUSG00000009575', 'ENSMUSG00000029177']

G1S_genes_Tirosh = ['MCM5', 'PCNA', 'TYMS', 'FEN1', 'MCM2', 'MCM4', 'RRM1', 'UNG', 'GINS2', 'MCM6', 'CDCA7', 'DTL', 'PRIM1', 'UHRF1', 'MLF1IP', 'HELLS', 'RFC2', 'RPA2', 'NASP', 'RAD51AP1', 'GMNN', 'WDR76', 'SLBP', 'CCNE2', 'UBR7', 'POLD3', 'MSH2', 'ATAD2', 'RAD51', 'RRM2', 'CDC45', 'CDC6', 'EXO1', 'TIPIN', 'DSCC1', 'BLM', 'CASP8AP2', 'USP1', 'CLSPN', 'POLA1', 'CHAF1B', 'BRIP1', 'E2F8']
G2M_genes_Tirosh = ['HMGB2', 'CDK1', 'NUSAP1', 'UBE2C', 'BIRC5', 'TPX2', 'TOP2A', 'NDC80', 'CKS2', 'NUF2', 'CKS1B', 'MKI67', 'TMPO', 'CENPF', 'TACC3', 'FAM64A', 'SMC4', 'CCNB2', 'CKAP2L', 'CKAP2', 'AURKB', 'BUB1', 'KIF11', 'ANP32E', 'TUBB4B', 'GTSE1', 'KIF20B', 'HJURP', 'CDCA3', 'HN1', 'CDC20', 'TTK', 'CDC25C', 'KIF2C', 'RANGAP1', 'NCAPD2', 'DLGAP5', 'CDCA2', 'CDCA8', 'ECT2', 'KIF23', 'HMMR', 'AURKA', 'PSRC1', 'ANLN', 'LBR', 'CKAP5', 'CENPE', 'CTCF', 'NEK2', 'G2E3', 'GAS2L3', 'CBX5', 'CENPA']
list_genes_fastCCsign = ['CDK1', 'UBE2C', 'TOP2A', 'TMPO', 'HJURP', 'RRM1', 'RAD51AP1', 'RRM2', 'CDC45', 'BLM', 'BRIP1', 'E2F8', 'HIST2H2AC']

G1S_genes_Tirosh = ['ENSG00000100297', 'ENSG00000132646', 'ENSG00000176890', 'ENSG00000168496', 'ENSG00000073111', 'ENSG00000104738', 'ENSG00000167325', 'ENSG00000076248', 'ENSG00000131153', 'ENSG00000076003', 'ENSG00000144354', 'ENSG00000143476', 'ENSG00000198056', 'ENSG00000276043', 'ENSG00000151725', 'ENSG00000119969', 'ENSG00000049541', 'ENSG00000117748', 'ENSG00000132780', 'ENSG00000111247', 'ENSG00000112312', 'ENSG00000092470', 'ENSG00000163950', 'ENSG00000175305', 'ENSG00000012963', 'ENSG00000077514', 'ENSG00000095002', 'ENSG00000156802', 'ENSG00000051180', 'ENSG00000171848', 'ENSG00000093009', 'ENSG00000094804', 'ENSG00000174371', 'ENSG00000075131', 'ENSG00000136982', 'ENSG00000197299', 'ENSG00000118412', 'ENSG00000162607', 'ENSG00000092853', 'ENSG00000101868', 'ENSG00000159259', 'ENSG00000136492', 'ENSG00000129173']
G2M_genes_Tirosh = ['ENSG00000164104', 'ENSG00000170312', 'ENSG00000137804', 'ENSG00000175063', 'ENSG00000089685', 'ENSG00000088325', 'ENSG00000131747', 'ENSG00000080986', 'ENSG00000123975', 'ENSG00000143228', 'ENSG00000173207', 'ENSG00000148773', 'ENSG00000120802', 'ENSG00000117724', 'ENSG00000013810', 'ENSG00000129195', 'ENSG00000113810', 'ENSG00000157456', 'ENSG00000169607', 'ENSG00000136108', 'ENSG00000178999', 'ENSG00000169679', 'ENSG00000138160', 'ENSG00000143401', 'ENSG00000188229', 'ENSG00000075218', 'ENSG00000138182', 'ENSG00000123485', 'ENSG00000111665', 'ENSG00000189159', 'ENSG00000117399', 'ENSG00000112742', 'ENSG00000158402', 'ENSG00000142945', 'ENSG00000100401', 'ENSG00000010292', 'ENSG00000126787', 'ENSG00000184661', 'ENSG00000134690', 'ENSG00000114346', 'ENSG00000137807', 'ENSG00000072571', 'ENSG00000087586', 'ENSG00000134222', 'ENSG00000011426', 'ENSG00000143815', 'ENSG00000175216', 'ENSG00000138778', 'ENSG00000102974', 'ENSG00000117650', 'ENSG00000092140', 'ENSG00000139354', 'ENSG00000094916', 'ENSG00000115163']
list_genes_fastCCsign = ['ENSG00000170312', 'ENSG00000175063', 'ENSG00000131747', 'ENSG00000120802', 'ENSG00000123485', 'ENSG00000167325', 'ENSG00000111247', 'ENSG00000171848', 'ENSG00000093009', 'ENSG00000197299', 'ENSG00000136492', 'ENSG00000129173', 'ENSG00000184260']




In [ ]:
IX = np.where(pd.Series(list_genes_ensembl_ids ).isin(G1S_genes_Tirosh) >0) [0]
len(IX), len( G1S_genes_Tirosh)

In [ ]:
%%time
v1 = f2[first_key]['block0_values'][:,IX].sum(axis = 1 )

In [ ]:
IX = np.where(pd.Series(list_genes_ensembl_ids ).isin(G2M_genes_Tirosh) >0) [0]
len(IX), len( G2M_genes_Tirosh)

In [ ]:
%%time
v2 = f2[first_key]['block0_values'][:,IX].sum(axis = 1 )
print( len(v2), type(v2), v2.shape) 

In [ ]:
sns.scatterplot(x = v1, y=v2)
plt.title(str_data_inf)
plt.show()

# G1/S-G2/M plot for day=1, cell_type = MkP (Megakaryocyte Progenitors)

In [ ]:
df_cite_train_y = pd.read_hdf('../input/open-problems-multimodal/train_cite_targets.h5')
genes = list(df_cite_train_y.keys())
df_cell_hue = df_cell.merge(df_cite_train_y, on = 'cell_id')
display(df_cell_hue.head(5))

In [ ]:
palette = 'rainbow'#'viridis'
n_x_subplots = 1
day = 7

mask = (df_cell_hue['day'] == day) & (df_cell_hue['cell_type'] == 'MkP' )                      
cell_ids = list(df_cell_hue['cell_id'][mask])

IX = np.where(pd.Series(list_genes_ensembl_ids ).isin(G1S_genes_Tirosh) >0) [0]
v1 = (f2[first_key]['block0_values'][:,IX]).sum(axis = 1 )
# Remark: Error - if write [IX0,IX] - "TypeError: Only one indexing vector or array is currently allowed for fancy indexing"
# so we proceed as above - in two steps

IX = np.where(pd.Series(list_genes_ensembl_ids ).isin(G2M_genes_Tirosh) >0) [0]
v2 = (f2[first_key]['block0_values'][:,IX]).sum(axis = 1 )

IX0 = np.where( pd.Series(list_cells_barcodes).isin(cell_ids) > 0)[0]


c = 0
for gene_id in genes: # ['day', 'donor', 'cell_type']:# , 'technology']:
    if c % n_x_subplots == 0:
        if c > 0:
            plt.show()
        fig = plt.figure(figsize = (20,5) ); c = 0
        plt.xlabel('G1/S score', fontsize = 20 )
        plt.ylabel('G2/M score', fontsize = 20 )

    c += 1
    fig.add_subplot(1,n_x_subplots ,c)    

    v = df_cell_hue[gene_id]

    ax = sns.scatterplot(x=v1[IX0], y=v2[IX0], hue=v[IX0], palette = palette )
    plt.setp(ax.get_legend().get_texts(), fontsize=10) # for legend text
    plt.setp(ax.get_legend().get_title(), fontsize=10) # for legend title    
    plt.title('Colored by '+gene_id, fontsize = 10)

#plt.title(str_data_inf + 'Day 4, MkP (Megakaryocyte Progenitors) ', fontsize = 20)

plt.show()


# G1/S-G2/M plot for day=2, cell_type = MkP (Megakaryocyte Progenitors)

Compare with: https://www.kaggle.com/code/alexandervc/mmscel-cell-cycle-03-daybydaychange-megakaryocyte?scriptVersionId=105307517&cellId=12



In [ ]:
palette = 'rainbow'#'viridis'
n_x_subplots = 4
day = 2

mask = (df_cell_hue['day'] == day) & (df_cell_hue['cell_type'] == 'MkP' )                      
cell_ids = list(df_cell_hue['cell_id'][mask])

IX = np.where(pd.Series(list_genes_ensembl_ids ).isin(G1S_genes_Tirosh) >0) [0]
v1 = (f2[first_key]['block0_values'][:,IX]).sum(axis = 1 )
# Remark: Error - if write [IX0,IX] - "TypeError: Only one indexing vector or array is currently allowed for fancy indexing"
# so we proceed as above - in two steps

IX = np.where(pd.Series(list_genes_ensembl_ids ).isin(G2M_genes_Tirosh) >0) [0]
v2 = (f2[first_key]['block0_values'][:,IX]).sum(axis = 1 )

IX0 = np.where( pd.Series(list_cells_barcodes).isin(cell_ids) > 0)[0]


c = 0
for gene_id in genes: # ['day', 'donor', 'cell_type']:# , 'technology']:
    if c % n_x_subplots == 0:
        if c > 0:
            plt.show()
        fig = plt.figure(figsize = (20,5) ); c = 0
        plt.xlabel('G1/S score', fontsize = 20 )
        plt.ylabel('G2/M score', fontsize = 20 )

    c += 1
    fig.add_subplot(1,n_x_subplots ,c)    

    v = df_cell_hue[gene_id]

    ax = sns.scatterplot(x=v1[IX0], y=v2[IX0], hue=v[IX0], palette = palette )
    plt.setp(ax.get_legend().get_texts(), fontsize=10) # for legend text
    plt.setp(ax.get_legend().get_title(), fontsize=10) # for legend title    
    plt.title('Colored by '+gene_id, fontsize = 10)

#plt.title(str_data_inf + 'Day 4, MkP (Megakaryocyte Progenitors) ', fontsize = 20)

plt.show()

# G1/S-G2/M plot for day=3, cell_type = MkP (Megakaryocyte Progenitors)

In [ ]:
palette = 'rainbow'#'viridis'
n_x_subplots = 4
day = 3

mask = (df_cell_hue['day'] == day) & (df_cell_hue['cell_type'] == 'MkP' )                      
cell_ids = list(df_cell_hue['cell_id'][mask])

IX = np.where(pd.Series(list_genes_ensembl_ids ).isin(G1S_genes_Tirosh) >0) [0]
v1 = (f2[first_key]['block0_values'][:,IX]).sum(axis = 1 )
# Remark: Error - if write [IX0,IX] - "TypeError: Only one indexing vector or array is currently allowed for fancy indexing"
# so we proceed as above - in two steps

IX = np.where(pd.Series(list_genes_ensembl_ids ).isin(G2M_genes_Tirosh) >0) [0]
v2 = (f2[first_key]['block0_values'][:,IX]).sum(axis = 1 )

IX0 = np.where( pd.Series(list_cells_barcodes).isin(cell_ids) > 0)[0]


c = 0
for gene_id in genes: # ['day', 'donor', 'cell_type']:# , 'technology']:
    if c % n_x_subplots == 0:
        if c > 0:
            plt.show()
        fig = plt.figure(figsize = (20,5) ); c = 0
        plt.xlabel('G1/S score', fontsize = 20 )
        plt.ylabel('G2/M score', fontsize = 20 )

    c += 1
    fig.add_subplot(1,n_x_subplots ,c)    

    v = df_cell_hue[gene_id]

    ax = sns.scatterplot(x=v1[IX0], y=v2[IX0], hue=v[IX0], palette = palette )
    plt.setp(ax.get_legend().get_texts(), fontsize=10) # for legend text
    plt.setp(ax.get_legend().get_title(), fontsize=10) # for legend title    
    plt.title('Colored by '+gene_id, fontsize = 10)

#plt.title(str_data_inf + 'Day 4, MkP (Megakaryocyte Progenitors) ', fontsize = 20)

plt.show()

# G1/S-G2/M plot for day=4, cell_type = MkP (Megakaryocyte Progenitors)

In [ ]:
palette = 'rainbow'#'viridis'
n_x_subplots = 4
day = 4

mask = (df_cell_hue['day'] == day) & (df_cell_hue['cell_type'] == 'MkP' )                      
cell_ids = list(df_cell_hue['cell_id'][mask])

IX = np.where(pd.Series(list_genes_ensembl_ids ).isin(G1S_genes_Tirosh) >0) [0]
v1 = (f2[first_key]['block0_values'][:,IX]).sum(axis = 1 )
# Remark: Error - if write [IX0,IX] - "TypeError: Only one indexing vector or array is currently allowed for fancy indexing"
# so we proceed as above - in two steps

IX = np.where(pd.Series(list_genes_ensembl_ids ).isin(G2M_genes_Tirosh) >0) [0]
v2 = (f2[first_key]['block0_values'][:,IX]).sum(axis = 1 )

IX0 = np.where( pd.Series(list_cells_barcodes).isin(cell_ids) > 0)[0]


c = 0
for gene_id in genes: # ['day', 'donor', 'cell_type']:# , 'technology']:
    if c % n_x_subplots == 0:
        if c > 0:
            plt.show()
        fig = plt.figure(figsize = (20,5) ); c = 0
        plt.xlabel('G1/S score', fontsize = 20 )
        plt.ylabel('G2/M score', fontsize = 20 )

    c += 1
    fig.add_subplot(1,n_x_subplots ,c)    

    v = df_cell_hue[gene_id]

    ax = sns.scatterplot(x=v1[IX0], y=v2[IX0], hue=v[IX0], palette = palette )
    plt.setp(ax.get_legend().get_texts(), fontsize=10) # for legend text
    plt.setp(ax.get_legend().get_title(), fontsize=10) # for legend title    
    plt.title('Colored by '+gene_id, fontsize = 10)

#plt.title(str_data_inf + 'Day 4, MkP (Megakaryocyte Progenitors) ', fontsize = 20)

plt.show()

# G1/S-G2/M plot for day=2, cell_type = MkP (Megakaryocyte Progenitors), Input data for hue


In [ ]:
# removed              'HLA-A-B-C': ['ENSG00000204525_HLA-C',
#              'ENSG00000206503_HLA-A',
#              'ENSG00000234745_HLA-B'],
cite_cols_important={'CD86': ['ENSG00000114013_CD86'],
             'CD274': ['ENSG00000120217_CD274'],
             'CD270': ['ENSG00000157873_TNFRSF14'],
             'CD155': ['ENSG00000073008_PVR'],
             'CD112': ['ENSG00000130202_NECTIN2'],
             'CD47': ['ENSG00000196776_CD47'],
             'CD48': ['ENSG00000117091_CD48'],
             'CD40': ['ENSG00000101017_CD40'],
             'CD154': ['ENSG00000102245_CD40LG'],
             'CD52': ['ENSG00000169442_CD52'],
             'CD3': ['ENSG00000167286_CD3D'],
             'CD8': [],
             'CD56': ['ENSG00000149294_NCAM1'],
             'CD19': ['ENSG00000177455_CD19'],
             'CD33': ['ENSG00000105383_CD33'],
             'CD11c': ['ENSG00000140678_ITGAX'],
             'CD45RA': ['ENSG00000081237_PTPRC'],
             'CD123': ['ENSG00000185291_IL3RA'],
             'CD7': ['ENSG00000173762_CD7'],
             'CD105': ['ENSG00000106991_ENG'],
             'CD49f': ['ENSG00000091409_ITGA6'],
             'CD194': ['ENSG00000183813_CCR4'],
             'CD4': ['ENSG00000010610_CD4'],
             'CD44': ['ENSG00000026508_CD44'],
             'CD14': ['ENSG00000170458_CD14'],
             'CD16': [],
             'CD25': ['ENSG00000134460_IL2RA'],
             'CD45RO': ['ENSG00000081237_PTPRC'],
             'CD279': [],
             'TIGIT': [],
             'Mouse-IgG1': [],
             'Mouse-IgG2a': [],
             'Mouse-IgG2b': [],
             'Rat-IgG2b': [],
             'CD20': ['ENSG00000156738_MS4A1'],
             'CD335': ['ENSG00000189430_NCR1'],
             'CD31': ['ENSG00000261371_PECAM1'],
             'Podoplanin': [],
             'CD146': ['ENSG00000076706_MCAM'],
             'IgM': ['ENSG00000211899_IGHM'],
             'CD5': [],
             'CD195': ['ENSG00000160791_CCR5'],
             'CD32': ['ENSG00000143226_FCGR2A'],
             'CD196': [],
             'CD185': ['ENSG00000160683_CXCR5'],
             'CD103': ['ENSG00000083457_ITGAE'],
             'CD69': ['ENSG00000110848_CD69'],
             'CD62L': ['ENSG00000188404_SELL'],
             'CD161': ['ENSG00000111796_KLRB1'],
             'CD152': [],
             'CD223': ['ENSG00000089692_LAG3'],
             'KLRG1': ['ENSG00000139187_KLRG1'],
             'CD27': ['ENSG00000139193_CD27'],
             'CD107a': ['ENSG00000185896_LAMP1'],
             'CD95': ['ENSG00000026103_FAS'],
             'CD134': ['ENSG00000186827_TNFRSF4'],
             'HLA-DR': ['ENSG00000204287_HLA-DRA'],
             'CD1c': ['ENSG00000158481_CD1C'],
             'CD11b': ['ENSG00000169896_ITGAM'],
             'CD64': ['ENSG00000150337_FCGR1A'],
             'CD141': ['ENSG00000178726_THBD'],
             'CD1d': ['ENSG00000158473_CD1D'],
             'CD314': [],
             'CD35': ['ENSG00000203710_CR1'],
             'CD57': [],
             'CD272': [],
             'CD278': ['ENSG00000163600_ICOS'],
             'CD58': ['ENSG00000116815_CD58'],
             'CD39': ['ENSG00000138185_ENTPD1'],
             'CX3CR1': ['ENSG00000168329_CX3CR1'],
             'CD24': ['ENSG00000272398_CD24'],
             'CD21': ['ENSG00000117322_CR2'],
             'CD11a': ['ENSG00000005844_ITGAL'],
             'CD79b': ['ENSG00000007312_CD79B'],
             'CD244': ['ENSG00000122223_CD244'],
             'CD169': [],
             'integrinB7': ['ENSG00000139626_ITGB7'],
             'CD268': ['ENSG00000159958_TNFRSF13C'],
             'CD42b': ['ENSG00000185245_GP1BA'],
             'CD54': ['ENSG00000090339_ICAM1'],
             'CD62P': ['ENSG00000174175_SELP'],
             'CD119': ['ENSG00000027697_IFNGR1'],
             'TCR': [],
             'Rat-IgG1': [],
             'Rat-IgG2a': [],
             'CD192': ['ENSG00000121807_CCR2'],
             'CD122': ['ENSG00000100385_IL2RB'],
             'FceRIa': ['ENSG00000179639_FCER1A'],
             'CD41': ['ENSG00000005961_ITGA2B'],
             'CD137': ['ENSG00000049249_TNFRSF9'],
             'CD163': ['ENSG00000177575_CD163'],
             'CD83': ['ENSG00000112149_CD83'],
             'CD124': ['ENSG00000077238_IL4R'],
             'CD13': ['ENSG00000166825_ANPEP'],
             'CD2': ['ENSG00000116824_CD2'],
             'CD226': ['ENSG00000150637_CD226'],
             'CD29': ['ENSG00000150093_ITGB1'],
             'CD303': ['ENSG00000198178_CLEC4C'],
             'CD49b': ['ENSG00000164171_ITGA2'],
             'CD81': ['ENSG00000110651_CD81'],
             'IgD': ['ENSG00000211898_IGHD'],
             'CD18': ['ENSG00000160255_ITGB2'],
             'CD28': [],
             'CD38': ['ENSG00000004468_CD38'],
             'CD127': ['ENSG00000168685_IL7R'],
             'CD45': ['ENSG00000081237_PTPRC'],
             'CD22': ['ENSG00000012124_CD22'],
             'CD71': ['ENSG00000072274_TFRC'],
             'CD26': ['ENSG00000197635_DPP4'],
             'CD115': ['ENSG00000182578_CSF1R'],
             'CD63': ['ENSG00000135404_CD63'],
             'CD304': ['ENSG00000099250_NRP1'],
             'CD36': ['ENSG00000135218_CD36'],
             'CD172a': ['ENSG00000198053_SIRPA'],
             'CD72': ['ENSG00000137101_CD72'],
             'CD158': [],
             'CD93': ['ENSG00000125810_CD93'],
             'CD49a': ['ENSG00000213949_ITGA1'],
             'CD49d': ['ENSG00000115232_ITGA4'],
             'CD73': [],
             'CD9': ['ENSG00000010278_CD9'],
             'TCRVa7.2': [],
             'TCRVd2': [],
             'LOX-1': ['ENSG00000173391_OLR1'],
             'CD158b': [],
             'CD158e1': [],
             'CD142': ['ENSG00000117525_F3'],
             'CD319': ['ENSG00000026751_SLAMF7'],
             'CD352': ['ENSG00000162739_SLAMF6'],
             'CD94': ['ENSG00000134539_KLRD1'],
             'CD162': ['ENSG00000110876_SELPLG'],
             'CD85j': ['ENSG00000104972_LILRB1'],
             'CD23': ['ENSG00000104921_FCER2'],
             'CD328': ['ENSG00000168995_SIGLEC7'],
             'HLA-E': ['ENSG00000204592_HLA-E'],
             'CD82': ['ENSG00000085117_CD82'],
             'CD101': ['ENSG00000134256_CD101'],
             'CD88': ['ENSG00000197405_C5AR1'],
             'CD224': ['ENSG00000100031_GGT1']}

In [ ]:
new_dict = dict([(value[0], key) for key, value in cite_cols_important.items() if value!=[]])
print(new_dict.values())
keys = list(new_dict.keys())+['cell_id','day','donor','cell_type','technology', 'gene_id']

In [ ]:
df_cite_train_y = pd.read_hdf(FP_CITE_TRAIN_INPUTS, axis = ['cell_id','gene_id'], usecols=keys)
df_cell_hue = df_cell.merge(df_cite_train_y, on = 'cell_id')
display(df_cell_hue.head(5))

In [ ]:
cols_to_be_deleted = [col for col in df_cell_hue.columns if col not in keys]
if len(cols_to_be_deleted)==21939:
    df_cell_hue.drop(cols_to_be_deleted, axis=1, inplace=True)
df_cell_hue.rename(columns = new_dict, inplace = True)
df_cell_hue.columns

In [ ]:
print(len(new_dict.values()))

In [ ]:
palette = 'rainbow'#'viridis'
n_x_subplots = 4
day = 2

mask = (df_cell_hue['day'] == day) & (df_cell_hue['cell_type'] == 'MkP' )                      
cell_ids = list(df_cell_hue['cell_id'][mask])

IX = np.where(pd.Series(list_genes_ensembl_ids ).isin(G1S_genes_Tirosh) >0) [0]
v1 = (f2[first_key]['block0_values'][:,IX]).sum(axis = 1 )
# Remark: Error - if write [IX0,IX] - "TypeError: Only one indexing vector or array is currently allowed for fancy indexing"
# so we proceed as above - in two steps

IX = np.where(pd.Series(list_genes_ensembl_ids ).isin(G2M_genes_Tirosh) >0) [0]
v2 = (f2[first_key]['block0_values'][:,IX]).sum(axis = 1 )

IX0 = np.where( pd.Series(list_cells_barcodes).isin(cell_ids) > 0)[0]


c = 0
for gene_id in new_dict.values(): # ['day', 'donor', 'cell_type']:# , 'technology']:
    if c % n_x_subplots == 0:
        if c > 0:
            plt.show()
        fig = plt.figure(figsize = (20,5) ); c = 0
        plt.xlabel('G1/S score', fontsize = 20 )
        plt.ylabel('G2/M score', fontsize = 20 )

    c += 1
    fig.add_subplot(1,n_x_subplots ,c)    

    v = df_cell_hue[gene_id]

    ax = sns.scatterplot(x=v1[IX0], y=v2[IX0], hue=v[IX0], palette = palette )
    plt.setp(ax.get_legend().get_texts(), fontsize=10) # for legend text
    plt.setp(ax.get_legend().get_title(), fontsize=10) # for legend title    
    plt.title('Colored by '+gene_id, fontsize = 10)

#plt.title(str_data_inf + 'Day 4, MkP (Megakaryocyte Progenitors) ', fontsize = 20)

plt.show()

# G1/S-G2/M plot for day=3, cell_type = MkP (Megakaryocyte Progenitors), Input data for hue


In [ ]:
palette = 'rainbow'#'viridis'
n_x_subplots = 4
day = 3

mask = (df_cell_hue['day'] == day) & (df_cell_hue['cell_type'] == 'MkP' )                      
cell_ids = list(df_cell_hue['cell_id'][mask])

IX = np.where(pd.Series(list_genes_ensembl_ids ).isin(G1S_genes_Tirosh) >0) [0]
v1 = (f2[first_key]['block0_values'][:,IX]).sum(axis = 1 )
# Remark: Error - if write [IX0,IX] - "TypeError: Only one indexing vector or array is currently allowed for fancy indexing"
# so we proceed as above - in two steps

IX = np.where(pd.Series(list_genes_ensembl_ids ).isin(G2M_genes_Tirosh) >0) [0]
v2 = (f2[first_key]['block0_values'][:,IX]).sum(axis = 1 )

IX0 = np.where( pd.Series(list_cells_barcodes).isin(cell_ids) > 0)[0]


c = 0
for gene_id in new_dict.values(): # ['day', 'donor', 'cell_type']:# , 'technology']:
    if c % n_x_subplots == 0:
        if c > 0:
            plt.show()
        fig = plt.figure(figsize = (20,5) ); c = 0
        plt.xlabel('G1/S score', fontsize = 20 )
        plt.ylabel('G2/M score', fontsize = 20 )

    c += 1
    fig.add_subplot(1,n_x_subplots ,c)    

    v = df_cell_hue[gene_id]

    ax = sns.scatterplot(x=v1[IX0], y=v2[IX0], hue=v[IX0], palette = palette )
    plt.setp(ax.get_legend().get_texts(), fontsize=10) # for legend text
    plt.setp(ax.get_legend().get_title(), fontsize=10) # for legend title    
    plt.title('Colored by '+gene_id, fontsize = 10)

#plt.title(str_data_inf + 'Day 4, MkP (Megakaryocyte Progenitors) ', fontsize = 20)

plt.show()

# G1/S-G2/M plot for day=4, cell_type = MkP (Megakaryocyte Progenitors), Input data for hue


In [ ]:
palette = 'rainbow'#'viridis'
n_x_subplots = 4
day = 4

mask = (df_cell_hue['day'] == day) & (df_cell_hue['cell_type'] == 'MkP' )                      
cell_ids = list(df_cell_hue['cell_id'][mask])

IX = np.where(pd.Series(list_genes_ensembl_ids ).isin(G1S_genes_Tirosh) >0) [0]
v1 = (f2[first_key]['block0_values'][:,IX]).sum(axis = 1 )
# Remark: Error - if write [IX0,IX] - "TypeError: Only one indexing vector or array is currently allowed for fancy indexing"
# so we proceed as above - in two steps

IX = np.where(pd.Series(list_genes_ensembl_ids ).isin(G2M_genes_Tirosh) >0) [0]
v2 = (f2[first_key]['block0_values'][:,IX]).sum(axis = 1 )

IX0 = np.where( pd.Series(list_cells_barcodes).isin(cell_ids) > 0)[0]


c = 0
for gene_id in new_dict.values(): # ['day', 'donor', 'cell_type']:# , 'technology']:
    if c % n_x_subplots == 0:
        if c > 0:
            plt.show()
        fig = plt.figure(figsize = (20,5) ); c = 0
        plt.xlabel('G1/S score', fontsize = 20 )
        plt.ylabel('G2/M score', fontsize = 20 )

    c += 1
    fig.add_subplot(1,n_x_subplots ,c)    

    v = df_cell_hue[gene_id]

    ax = sns.scatterplot(x=v1[IX0], y=v2[IX0], hue=v[IX0], palette = palette )
    plt.setp(ax.get_legend().get_texts(), fontsize=10) # for legend text
    plt.setp(ax.get_legend().get_title(), fontsize=10) # for legend title    
    plt.title('Colored by '+gene_id, fontsize = 10)

#plt.title(str_data_inf + 'Day 4, MkP (Megakaryocyte Progenitors) ', fontsize = 20)

plt.show()

# Loop over all cell types 

In [ ]:
dict_cell_types = { 'MasP': 'Mast Cell Progenitor',
'MkP': 'Megakaryocyte Progenitor',
'NeuP': 'Neutrophil Progenitor',
'MoP': 'Monocyte Progenitor',
'EryP': 'Erythrocyte Progenitor',
'HSC': 'Hematoploetic Stem Cell',
'BP': 'B-Cell Progenitor'}

In [ ]:
df_cell.head(1)
d = pd.DataFrame(index =list_cells_barcodes,  )
d = d.join(df_cell.set_index('cell_id'), how = 'left')
d['str_donor'] = d['donor'].apply(lambda x: str(x))
d

In [ ]:
for cell_type in ['MkP','HSC',  'EryP', 'NeuP', 'MasP' ,  'MoP' , 'BP' ]:
    print();print();print()
    print(cell_type)
    
    for day in [2,3,4,7]:

        mask = (df_cell['day'] == day) & (df_cell['cell_type'] == cell_type )                      
        #print('Day',day, mask.sum() )
        cell_ids = list(df_cell['cell_id'][mask])
        #print(len(cell_ids) , cell_ids[:10])

        IX0 = np.where( pd.Series(list_cells_barcodes).isin(cell_ids) > 0)[0]
        print('Day', day,mask.sum(), len(IX0))# , IX0[:10]) # , len(cell_ids ))

        plt.figure(figsize = (20,10))
        if d['str_donor'].iloc[IX0].nunique() > 1:
            sns.scatterplot(x = v1[IX0], y=v2[IX0] , hue = d['str_donor'].iloc[IX0], palette = 'rainbow')#  ['blue'])
        else:
            sns.scatterplot(x = v1[IX0], y=v2[IX0] , hue = d['str_donor'].iloc[IX0], palette =  ['blue'])
            
        plt.xlabel('G1/S score', fontsize = 20 )
        plt.ylabel('G2/M score', fontsize = 20 )
        plt.title(str_data_inf + '  Day '+str(day) + ', '+ cell_type +  ' ' + dict_cell_types[cell_type] + ' n_cells=%d'%len(IX0) , fontsize = 20)

        plt.show()

In [ ]:
print('%.1f seconds passed total '%(time.time()-t0start) )
